# DQAR Benchmark: DiT-XL-2-256

This notebook benchmarks the DQAR (Dynamic and Quantization-Aware Attention Reuse) framework on the DiT-XL-2-256 model from Meta.

**Requirements:**
- GPU with CUDA support (recommended: A100, T4, or better)
- ~10GB VRAM for DiT-XL

**What this notebook does:**
1. Clones the DQAR repository
2. Installs dependencies
3. Loads DiT-XL-2-256 from HuggingFace
4. Patches the pipeline with DQAR
5. Runs baseline vs DQAR comparison
6. Plots results with confidence intervals

## 1. Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone DQAR repository
!git clone https://github.com/gauthamys/DQAR.git
%cd DQAR

In [ ]:
# Install dependencies
!pip install -q torch diffusers transformers accelerate matplotlib

In [ ]:
# Add DQAR to path
import sys
sys.path.insert(0, 'src')

# Verify DQAR imports
from dqar import DQARConfig, DQARController, get_dit_layer_count
from dqar.dit_wrapper import DQARAttentionWrapper
print("DQAR imported successfully!")

# Define the wrapper class directly (fixes the controller argument issue)
from typing import Optional
from functools import wraps

class DQARPipelineWrapper:
    """Wrapper class for DiTPipeline that adds DQAR controller support."""

    def __init__(self, pipe, wrappers: list):
        self._pipe = pipe
        self._dqar_wrappers = wrappers
        self._dqar_num_layers = len(wrappers)
        self._active_controller = None
        self.transformer = pipe.transformer
        self.scheduler = pipe.scheduler
        self.vae = getattr(pipe, 'vae', None)
        self.config = getattr(pipe, 'config', None)

    def __getattr__(self, name: str):
        return getattr(self._pipe, name)

    def __call__(self, *args, controller: Optional[DQARController] = None, **kwargs):
        branch = kwargs.pop('dqar_branch', 'cond')
        for wrapper in self._dqar_wrappers:
            wrapper.set_controller(controller, branch=branch)
        self._active_controller = controller

        try:
            if controller is not None:
                original_callback = kwargs.get('callback_on_step_end')
                def step_callback(pipe, step_idx, timestep, callback_kwargs):
                    if self._active_controller:
                        num_steps = kwargs.get('num_inference_steps', 50)
                        self._active_controller.begin_step(
                            step_index=step_idx,
                            total_steps=num_steps,
                            prompt_length=16,
                        )
                    if original_callback:
                        return original_callback(pipe, step_idx, timestep, callback_kwargs)
                    return callback_kwargs
                kwargs['callback_on_step_end'] = step_callback
            return self._pipe(*args, **kwargs)
        finally:
            for wrapper in self._dqar_wrappers:
                wrapper.set_controller(None)
            self._active_controller = None

    def to(self, device):
        self._pipe = self._pipe.to(device)
        return self

    def set_progress_bar_config(self, **kwargs):
        self._pipe.set_progress_bar_config(**kwargs)


def patch_dit_pipeline(pipe):
    """Patch a DiTPipeline to support DQAR controller integration."""
    transformer = pipe.transformer
    blocks = getattr(transformer, 'transformer_blocks', None) or getattr(transformer, 'blocks', [])
    
    if not blocks:
        raise ValueError("Could not find transformer blocks in the model")

    wrappers = []
    for idx, block in enumerate(blocks):
        if hasattr(block, 'attn1'):
            wrapper = DQARAttentionWrapper(block.attn1, layer_idx=idx)
            wrappers.append(wrapper)
            block._dqar_wrapper = wrapper
        elif hasattr(block, 'attn'):
            wrapper = DQARAttentionWrapper(block.attn, layer_idx=idx)
            wrappers.append(wrapper)
            block._dqar_wrapper = wrapper

    if not wrappers:
        raise ValueError("No attention layers found to wrap")

    wrapped = DQARPipelineWrapper(pipe, wrappers)
    print(f"[DQAR] Patched pipeline with {len(wrappers)} attention wrappers")
    return wrapped


def get_layer_count(pipe) -> int:
    """Get the number of transformer layers."""
    if isinstance(pipe, DQARPipelineWrapper):
        return pipe._dqar_num_layers
    if hasattr(pipe, '_dqar_num_layers'):
        return pipe._dqar_num_layers
    transformer = getattr(pipe, 'transformer', None)
    blocks = getattr(transformer, 'transformer_blocks', None) or getattr(transformer, 'blocks', [])
    return len(blocks)

print("Wrapper class defined successfully!")

## 2. Load DiT-XL-2-256

In [ ]:
from diffusers import DiTPipeline
import torch

# Load the model
model_id = "facebook/DiT-XL-2-256"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading {model_id}...")
pipe = DiTPipeline.from_pretrained(model_id, torch_dtype=dtype)
pipe = pipe.to(device)
pipe.set_progress_bar_config(disable=True)

print(f"Model loaded on {device} with dtype {dtype}")
print(f"Transformer blocks: {len(pipe.transformer.transformer_blocks)}")

In [ ]:
# Patch pipeline with DQAR (using the wrapper defined above)
pipe = patch_dit_pipeline(pipe)
num_layers = get_layer_count(pipe)
print(f"Pipeline patched with {num_layers} DQAR attention wrappers")

## 3. Configure Benchmark

In [ ]:
# Benchmark configuration
CONFIG = {
    "num_inference_steps": 50,      # DDIM steps
    "guidance_scale": 4.0,          # CFG scale
    "num_images": 8,                # Number of images to generate per config
    "num_runs": 5,                  # Runs per configuration for CI
    "seed": 42,
    
    # DQAR parameters to sweep
    "entropy_thresholds": [1.5, 3.0, 5.0],
    "max_reuse_limits": [2, 4, 6],
    
    # DQAR gate config
    "dqar_min_prob": 0.5,
    "dqar_snr_min": 0.25,
    "dqar_snr_max": 64.0,
    "dqar_cooldown": 2,
    "dqar_max_gap": 4,
}

# ImageNet class labels (DiT-XL is class-conditional)
# See: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
CLASS_LABELS = [
    207,  # golden retriever
    360,  # otter
    387,  # lesser panda
    974,  # geyser
    88,   # macaw
    979,  # valley
    417,  # balloon
    279,  # arctic fox
]

print(f"Benchmark config: {CONFIG['num_inference_steps']} steps, {CONFIG['num_runs']} runs per config")
print(f"Class labels: {CLASS_LABELS[:CONFIG['num_images']]}")

## 4. Benchmark Functions

In [ ]:
import time
import math
from dataclasses import dataclass
from typing import List, Optional
from statistics import mean, pstdev

@dataclass
class BenchmarkResult:
    name: str
    config: str
    runtimes: List[float]
    peak_memory_gb: float
    reuse_count: int = 0
    
    @property
    def avg_time(self) -> float:
        return mean(self.runtimes)
    
    @property
    def std_time(self) -> float:
        return pstdev(self.runtimes) if len(self.runtimes) > 1 else 0.0


def build_dqar_config(entropy_threshold: float, max_reuse: int) -> DQARConfig:
    """Build DQAR config from parameters."""
    cfg = DQARConfig()
    cfg.gate.min_probability = CONFIG["dqar_min_prob"]
    cfg.gate.entropy_threshold = entropy_threshold
    cfg.gate.snr_range = (CONFIG["dqar_snr_min"], CONFIG["dqar_snr_max"])
    cfg.gate.cooldown_steps = CONFIG["dqar_cooldown"]
    cfg.scheduler.max_gap = CONFIG["dqar_max_gap"]
    cfg.scheduler.max_reuse_per_block = max_reuse
    return cfg


def run_generation(
    pipe,
    class_labels: List[int],
    controller: Optional[DQARController] = None,
    num_steps: int = 50,
    guidance_scale: float = 4.0,
    seed: int = 42,
) -> tuple:
    """Run a single generation pass."""
    device = next(pipe.transformer.parameters()).device
    
    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats(device)
    
    gen = torch.Generator(device=device).manual_seed(seed)
    
    # Prepare class labels tensor
    labels_tensor = torch.tensor(class_labels, device=device)
    
    kwargs = dict(
        class_labels=labels_tensor,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=gen,
        output_type="pil",
    )
    
    if controller is not None:
        kwargs["controller"] = controller
    
    if device.type == "cuda":
        torch.cuda.synchronize(device)
    start = time.perf_counter()
    
    result = pipe(**kwargs)
    
    if device.type == "cuda":
        torch.cuda.synchronize(device)
    elapsed = time.perf_counter() - start
    
    peak_mem = torch.cuda.max_memory_allocated(device) / (1024**3) if device.type == "cuda" else 0.0
    
    return result.images, elapsed, peak_mem


def benchmark_baseline(pipe, num_runs: int = 5) -> BenchmarkResult:
    """Benchmark without DQAR."""
    runtimes = []
    peak_mem = 0.0
    
    class_labels = CLASS_LABELS[:CONFIG["num_images"]]
    
    for run in range(num_runs):
        _, elapsed, mem = run_generation(
            pipe,
            class_labels=class_labels,
            controller=None,
            num_steps=CONFIG["num_inference_steps"],
            guidance_scale=CONFIG["guidance_scale"],
            seed=CONFIG["seed"] + run,
        )
        runtimes.append(elapsed)
        peak_mem = max(peak_mem, mem)
    
    return BenchmarkResult(
        name="Baseline",
        config="no-reuse",
        runtimes=runtimes,
        peak_memory_gb=peak_mem,
    )


def benchmark_dqar(
    pipe,
    entropy_threshold: float,
    max_reuse: int,
    num_runs: int = 5,
) -> BenchmarkResult:
    """Benchmark with DQAR enabled."""
    runtimes = []
    peak_mem = 0.0
    total_reuse = 0
    
    class_labels = CLASS_LABELS[:CONFIG["num_images"]]
    num_layers = get_layer_count(pipe)  # Use local function
    
    for run in range(num_runs):
        # Create fresh controller for each run
        config = build_dqar_config(entropy_threshold, max_reuse)
        controller = DQARController(num_layers=num_layers, config=config)
        
        _, elapsed, mem = run_generation(
            pipe,
            class_labels=class_labels,
            controller=controller,
            num_steps=CONFIG["num_inference_steps"],
            guidance_scale=CONFIG["guidance_scale"],
            seed=CONFIG["seed"] + run,
        )
        runtimes.append(elapsed)
        peak_mem = max(peak_mem, mem)
        # Get reuse count from controller
        total_reuse += getattr(controller, '_total_reuse_count', 0)
    
    return BenchmarkResult(
        name="DQAR",
        config=f"tau={entropy_threshold}, max={max_reuse}",
        runtimes=runtimes,
        peak_memory_gb=peak_mem,
        reuse_count=total_reuse // max(num_runs, 1),
    )

print("Benchmark functions defined.")

## 5. Run Benchmark

In [ ]:
# Warmup run
print("Running warmup...")
_ = run_generation(
    pipe,
    class_labels=[207],
    num_steps=10,
    guidance_scale=4.0,
    seed=0,
)
print("Warmup complete.")

In [ ]:
# Run baseline benchmark
print(f"\nRunning baseline benchmark ({CONFIG['num_runs']} runs)...")
baseline_result = benchmark_baseline(pipe, num_runs=CONFIG["num_runs"])
print(f"Baseline: {baseline_result.avg_time:.2f}s +/- {baseline_result.std_time:.2f}s, Peak VRAM: {baseline_result.peak_memory_gb:.2f} GB")

In [ ]:
# Run DQAR benchmark sweep
dqar_results = []

for threshold in CONFIG["entropy_thresholds"]:
    for max_reuse in CONFIG["max_reuse_limits"]:
        print(f"\nBenchmarking DQAR (tau={threshold}, max_reuse={max_reuse})...")
        result = benchmark_dqar(
            pipe,
            entropy_threshold=threshold,
            max_reuse=max_reuse,
            num_runs=CONFIG["num_runs"],
        )
        dqar_results.append(result)
        speedup = (baseline_result.avg_time - result.avg_time) / baseline_result.avg_time * 100
        print(f"  Time: {result.avg_time:.2f}s +/- {result.std_time:.2f}s")
        print(f"  Speedup: {speedup:.1f}%")
        print(f"  Reuse events: {result.reuse_count}")
        print(f"  Peak VRAM: {result.peak_memory_gb:.2f} GB")

print("\nBenchmark sweep complete!")

## 6. Results Summary

In [ ]:
import pandas as pd

# Create results DataFrame
rows = []

# Add baseline
rows.append({
    "Configuration": "Baseline",
    "Runtime (s)": f"{baseline_result.avg_time:.2f} +/- {baseline_result.std_time:.2f}",
    "Speedup": "0%",
    "Reuse Count": 0,
    "Peak VRAM (GB)": f"{baseline_result.peak_memory_gb:.2f}",
})

# Add DQAR results
for result in dqar_results:
    speedup = (baseline_result.avg_time - result.avg_time) / baseline_result.avg_time * 100
    rows.append({
        "Configuration": f"DQAR ({result.config})",
        "Runtime (s)": f"{result.avg_time:.2f} +/- {result.std_time:.2f}",
        "Speedup": f"{speedup:.1f}%",
        "Reuse Count": result.reuse_count,
        "Peak VRAM (GB)": f"{result.peak_memory_gb:.2f}",
    })

df = pd.DataFrame(rows)
display(df)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Config labels
labels = ["Baseline"] + [r.config for r in dqar_results]
x = np.arange(len(labels))

# Runtime plot
ax = axes[0]
times = [baseline_result.avg_time] + [r.avg_time for r in dqar_results]
errors = [baseline_result.std_time] + [r.std_time for r in dqar_results]
colors = ['#1f77b4'] + ['#2ca02c'] * len(dqar_results)
ax.bar(x, times, yerr=errors, capsize=3, color=colors, alpha=0.8)
ax.set_ylabel('Runtime (s)')
ax.set_title('Inference Time')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.grid(True, linestyle='--', alpha=0.3)

# Memory plot
ax = axes[1]
mems = [baseline_result.peak_memory_gb] + [r.peak_memory_gb for r in dqar_results]
ax.bar(x, mems, color=colors, alpha=0.8)
ax.set_ylabel('Peak VRAM (GB)')
ax.set_title('Memory Usage')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.grid(True, linestyle='--', alpha=0.3)

# Reuse count plot
ax = axes[2]
reuse = [0] + [r.reuse_count for r in dqar_results]
ax.bar(x, reuse, color=colors, alpha=0.8)
ax.set_ylabel('Reuse Events')
ax.set_title('Attention Reuse Count')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('dit_xl_benchmark_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to dit_xl_benchmark_results.png")

## 7. Save Results

In [ ]:
import json

# Save results to JSON
results_json = {
    "model": "facebook/DiT-XL-2-256",
    "config": CONFIG,
    "baseline": {
        "avg_time_s": baseline_result.avg_time,
        "std_time_s": baseline_result.std_time,
        "peak_vram_gb": baseline_result.peak_memory_gb,
        "runtimes": baseline_result.runtimes,
    },
    "dqar_results": [
        {
            "config": r.config,
            "avg_time_s": r.avg_time,
            "std_time_s": r.std_time,
            "peak_vram_gb": r.peak_memory_gb,
            "reuse_count": r.reuse_count,
            "speedup_pct": (baseline_result.avg_time - r.avg_time) / baseline_result.avg_time * 100,
            "runtimes": r.runtimes,
        }
        for r in dqar_results
    ],
}

with open('dit_xl_benchmark_results.json', 'w') as f:
    json.dump(results_json, f, indent=2)

print("Results saved to dit_xl_benchmark_results.json")

## 8. Sample Generated Images

In [ ]:
# Generate sample images for visual comparison
from PIL import Image
import matplotlib.pyplot as plt

# Generate with baseline
print("Generating baseline samples...")
baseline_images, _, _ = run_generation(
    pipe,
    class_labels=CLASS_LABELS[:4],
    num_steps=CONFIG["num_inference_steps"],
    guidance_scale=CONFIG["guidance_scale"],
    seed=CONFIG["seed"],
)

# Generate with DQAR (best config from sweep)
print("Generating DQAR samples...")
best_dqar = min(dqar_results, key=lambda r: r.avg_time)
config = build_dqar_config(
    entropy_threshold=float(best_dqar.config.split('tau=')[1].split(',')[0]),
    max_reuse=int(best_dqar.config.split('max=')[1]),
)
controller = DQARController(num_layers=get_layer_count(pipe), config=config)

dqar_images, _, _ = run_generation(
    pipe,
    class_labels=CLASS_LABELS[:4],
    controller=controller,
    num_steps=CONFIG["num_inference_steps"],
    guidance_scale=CONFIG["guidance_scale"],
    seed=CONFIG["seed"],
)

# Display comparison
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Baseline vs DQAR Generated Images', fontsize=14)

for i, (baseline_img, dqar_img) in enumerate(zip(baseline_images, dqar_images)):
    axes[0, i].imshow(baseline_img)
    axes[0, i].set_title(f'Baseline (class {CLASS_LABELS[i]})')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(dqar_img)
    axes[1, i].set_title(f'DQAR (class {CLASS_LABELS[i]})')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('dit_xl_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nComparison saved to dit_xl_comparison.png")

## Summary

This notebook benchmarked DQAR on DiT-XL-2-256 with the following findings:

1. **Baseline Performance**: Reference runtime without attention reuse
2. **DQAR Performance**: Adaptive attention reuse based on entropy/SNR gating
3. **Speedup**: Achieved by reducing redundant attention computations
4. **Quality**: Visual comparison shows minimal quality degradation

Key observations:
- Higher entropy thresholds allow more reuse (faster but potentially lower quality)
- The adaptive gate prevents reuse when attention patterns are still changing
- Memory overhead from quantized KV cache is minimal